Spark Iris Intro

Running Spark in a Kaggle Notebook

a modified version of: 

https://www.kaggle.com/kkhandekar/apache-spark-beginner-tutorial

In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/housing-prices-dataset/Housing.csv
/kaggle/input/kingcountyhousing/KC_housing_data.csv
/kaggle/input/final-house/house.csv


In [27]:
# before you can use a data set, it has to be in your data space, see the browser on the upper right side

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/housing-prices-dataset/Housing.csv
/kaggle/input/kingcountyhousing/KC_housing_data.csv
/kaggle/input/final-house/house.csv


Install pyspark in this environment

In [28]:
#install Apache Spark
!pip install pyspark --quiet

Install packages

Most "standard" packages seem to be available on Kaggle

In [30]:
#Generic Libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#Apache Spark Libraries
import pyspark
from pyspark.sql import SparkSession

#Apache Spark ML CLassifier Libraries
from pyspark.ml.classification import DecisionTreeClassifier,RandomForestClassifier,NaiveBayes

#Apache Spark Evaluation Library
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#Apache Spark Features libraries
from pyspark.ml.feature import StandardScaler,StringIndexer, VectorAssembler, VectorIndexer, OneHotEncoder

#Apache Spark Pipelin Library
from pyspark.ml import Pipeline

# Apache Spark `DenseVector`
from pyspark.ml.linalg import DenseVector

#Data Split Libraries
import sklearn
from sklearn.model_selection import train_test_split


#Tabulating Data
from tabulate import tabulate

#Garbage
import gc

In [31]:
#Building Spark Session
spark = (SparkSession.builder
                  .appName('Apache Spark Beginner Tutorial')
                  .config("spark.executor.memory", "1G")
                  .config("spark.executor.cores","4")
                  .getOrCreate())

24/03/26 16:08:16 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


I'm not sure just what the setLogLevel function below does,   It may make sense to turn off this operation

In [32]:
spark.sparkContext.setLogLevel('INFO')

Check the Spark version level,   this looks straightforward

In [33]:
spark.version

'3.5.1'

In [38]:
#This is just a test loading of a kaggle data set into a pandas data set
# You have to have the data sets in your kaggle directory to load them,  there is a "Data" load data option on the upper right side of
# the kaggle notebook system

df = pd.read_csv('/kaggle/input/iris/Iris.csv')

In [39]:
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


Okay, the basic data load operation worked

Now, we will load the data set into spark.   The load function looks fairly standard, note the syntax

In [40]:
url = '/kaggle/input/iris/Iris.csv'

data = spark.read.format("csv") \
       .option("header", "true") \
       .option("inferSchema","true")\
       .load(url) 

data.cache() #for faster re-use

24/03/26 16:13:10 INFO InMemoryFileIndex: It took 2 ms to list leaf files for 1 paths.
24/03/26 16:13:10 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.
24/03/26 16:13:10 INFO FileSourceStrategy: Pushed Filters: 
24/03/26 16:13:10 INFO FileSourceStrategy: Post-Scan Filters: (length(trim(value#4598, None)) > 0)
24/03/26 16:13:10 INFO MemoryStore: Block broadcast_23 stored as values in memory (estimated size 200.1 KiB, free 433.2 MiB)
24/03/26 16:13:11 INFO MemoryStore: Block broadcast_23_piece0 stored as bytes in memory (estimated size 34.4 KiB, free 433.2 MiB)
24/03/26 16:13:11 INFO BlockManagerInfo: Added broadcast_23_piece0 in memory on b2edbf16b081:40551 (size: 34.4 KiB, free: 433.9 MiB)
24/03/26 16:13:11 INFO SparkContext: Created broadcast 23 from load at NativeMethodAccessorImpl.java:0
24/03/26 16:13:11 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/03/26 16:13:11 INFO Sp

DataFrame[Id: int, SepalLengthCm: double, SepalWidthCm: double, PetalLengthCm: double, PetalWidthCm: double, Species: string]

In [41]:
#Total records 
data.count()

24/03/26 16:13:17 INFO MemoryStore: Block broadcast_27 stored as values in memory (estimated size 200.0 KiB, free 432.7 MiB)
24/03/26 16:13:17 INFO MemoryStore: Block broadcast_27_piece0 stored as bytes in memory (estimated size 34.4 KiB, free 432.7 MiB)
24/03/26 16:13:17 INFO BlockManagerInfo: Added broadcast_27_piece0 in memory on b2edbf16b081:40551 (size: 34.4 KiB, free: 433.8 MiB)
24/03/26 16:13:17 INFO SparkContext: Created broadcast 27 from count at NativeMethodAccessorImpl.java:0
24/03/26 16:13:17 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/03/26 16:13:17 INFO DAGScheduler: Got job 22 (count at NativeMethodAccessorImpl.java:0) with 1 output partitions
24/03/26 16:13:17 INFO DAGScheduler: Final stage: ResultStage 26 (count at NativeMethodAccessorImpl.java:0)
24/03/26 16:13:17 INFO DAGScheduler: Parents of final stage: List()
24/03/26 16:13:17 INFO DAGScheduler: Missing parents: List()
24/0

150

We can take a look at how Spark is storing the data internally,  note the strong similarity to a SQL style data table schema

The data storage in Spark looks a lot like an SQL database,   Spark is meant to work with distributed databases,  particularly Apache Hive

In [42]:
#Data Type
data.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)



data.show() works like .head() does,   it shows the data table, very much in database style report

In [43]:
#Display records
data.show(5)

+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
+---+-------------+------------+-------------+------------+-----------+
only showing top 5 rows



24/03/26 16:13:29 INFO CodeGenerator: Code generated in 12.253547 ms
24/03/26 16:13:29 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
24/03/26 16:13:29 INFO DAGScheduler: Got job 25 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
24/03/26 16:13:29 INFO DAGScheduler: Final stage: ResultStage 30 (showString at NativeMethodAccessorImpl.java:0)
24/03/26 16:13:29 INFO DAGScheduler: Parents of final stage: List()
24/03/26 16:13:29 INFO DAGScheduler: Missing parents: List()
24/03/26 16:13:29 INFO DAGScheduler: Submitting ResultStage 30 (MapPartitionsRDD[118] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
24/03/26 16:13:29 INFO MemoryStore: Block broadcast_31 stored as values in memory (estimated size 22.2 KiB, free 432.6 MiB)
24/03/26 16:13:29 INFO MemoryStore: Block broadcast_31_piece0 stored as bytes in memory (estimated size 9.9 KiB, free 432.5 MiB)
24/03/26 16:13:29 INFO BlockManagerInfo: Added broad

Here is a groupby operation

In [44]:
#Records per Species
data.groupBy('species').count().show()

24/03/26 16:13:33 INFO CodeGenerator: Code generated in 30.492688 ms
24/03/26 16:13:33 INFO DAGScheduler: Registering RDD 123 (showString at NativeMethodAccessorImpl.java:0) as input to shuffle 5
24/03/26 16:13:33 INFO DAGScheduler: Got map stage job 26 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
24/03/26 16:13:33 INFO DAGScheduler: Final stage: ShuffleMapStage 31 (showString at NativeMethodAccessorImpl.java:0)
24/03/26 16:13:33 INFO DAGScheduler: Parents of final stage: List()
24/03/26 16:13:33 INFO DAGScheduler: Missing parents: List()
24/03/26 16:13:33 INFO DAGScheduler: Submitting ShuffleMapStage 31 (MapPartitionsRDD[123] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
24/03/26 16:13:33 INFO MemoryStore: Block broadcast_32 stored as values in memory (estimated size 42.5 KiB, free 433.5 MiB)
24/03/26 16:13:33 INFO MemoryStore: Block broadcast_32_piece0 stored as bytes in memory (estimated size 19.6 KiB, free 433.5 MiB)
24/

+---------------+-----+
|        species|count|
+---------------+-----+
| Iris-virginica|   50|
|    Iris-setosa|   50|
|Iris-versicolor|   50|
+---------------+-----+



24/03/26 16:13:33 INFO CodeGenerator: Code generated in 15.625202 ms
24/03/26 16:13:33 INFO Executor: Finished task 0.0 in stage 33.0 (TID 32). 5138 bytes result sent to driver
24/03/26 16:13:33 INFO TaskSetManager: Finished task 0.0 in stage 33.0 (TID 32) in 39 ms on b2edbf16b081 (executor driver) (1/1)
24/03/26 16:13:33 INFO TaskSchedulerImpl: Removed TaskSet 33.0, whose tasks have all completed, from pool 
24/03/26 16:13:33 INFO DAGScheduler: ResultStage 33 (showString at NativeMethodAccessorImpl.java:0) finished in 0.051 s
24/03/26 16:13:33 INFO DAGScheduler: Job 27 is finished. Cancelling potential speculative or zombie tasks for this job
24/03/26 16:13:33 INFO TaskSchedulerImpl: Killing all running tasks in stage 33: Stage finished
24/03/26 16:13:33 INFO DAGScheduler: Job 27 finished: showString at NativeMethodAccessorImpl.java:0, took 0.058303 s


Here is a describe() function,  that does the typical data summary operation

In [45]:
#Dataset Summary Stats
data.describe().show()

24/03/26 16:13:42 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/03/26 16:13:42 INFO DAGScheduler: Registering RDD 131 (showString at NativeMethodAccessorImpl.java:0) as input to shuffle 6
24/03/26 16:13:42 INFO DAGScheduler: Got map stage job 28 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
24/03/26 16:13:42 INFO DAGScheduler: Final stage: ShuffleMapStage 34 (showString at NativeMethodAccessorImpl.java:0)
24/03/26 16:13:42 INFO DAGScheduler: Parents of final stage: List()
24/03/26 16:13:42 INFO DAGScheduler: Missing parents: List()
24/03/26 16:13:42 INFO DAGScheduler: Submitting ShuffleMapStage 34 (MapPartitionsRDD[131] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
24/03/26 16:13:42 INFO MemoryStore: Block broadcast_34 stored as values in memory (estimated size 50.6 KiB, free 433.4 MiB)
24/03/26 16:1

+-------+------------------+------------------+-------------------+------------------+------------------+--------------+
|summary|                Id|     SepalLengthCm|       SepalWidthCm|     PetalLengthCm|      PetalWidthCm|       Species|
+-------+------------------+------------------+-------------------+------------------+------------------+--------------+
|  count|               150|               150|                150|               150|               150|           150|
|   mean|              75.5| 5.843333333333335| 3.0540000000000007|3.7586666666666693|1.1986666666666672|          NULL|
| stddev|43.445367992456916|0.8280661279778637|0.43359431136217375| 1.764420419952262|0.7631607417008414|          NULL|
|    min|                 1|               4.3|                2.0|               1.0|               0.1|   Iris-setosa|
|    max|               150|               7.9|                4.4|               6.9|               2.5|Iris-virginica|
+-------+------------------+----

24/03/26 16:13:42 INFO CodeGenerator: Code generated in 59.797185 ms
24/03/26 16:13:42 INFO CodeGenerator: Code generated in 28.361492 ms
24/03/26 16:13:42 INFO CodeGenerator: Code generated in 43.479029 ms
24/03/26 16:13:42 INFO CodeGenerator: Code generated in 15.96853 ms
24/03/26 16:13:42 INFO Executor: Finished task 0.0 in stage 36.0 (TID 34). 4765 bytes result sent to driver
24/03/26 16:13:42 INFO TaskSetManager: Finished task 0.0 in stage 36.0 (TID 34) in 302 ms on b2edbf16b081 (executor driver) (1/1)
24/03/26 16:13:42 INFO TaskSchedulerImpl: Removed TaskSet 36.0, whose tasks have all completed, from pool 
24/03/26 16:13:42 INFO DAGScheduler: ResultStage 36 (showString at NativeMethodAccessorImpl.java:0) finished in 0.350 s
24/03/26 16:13:42 INFO DAGScheduler: Job 29 is finished. Cancelling potential speculative or zombie tasks for this job
24/03/26 16:13:42 INFO TaskSchedulerImpl: Killing all running tasks in stage 36: Stage finished
24/03/26 16:13:42 INFO DAGScheduler: Job 29 f

The species names are text, and we know ML models need to have integer codings

The StringIndexer in Spark takes care of this

In [47]:
#String Indexing the Species column
SIndexer = StringIndexer(inputCol='Species', outputCol='species_indx')
data = SIndexer.fit(data).transform(data)

#Inspect the dataset
data.show(5)

24/03/26 16:14:10 INFO DAGScheduler: Registering RDD 140 (collect at StringIndexer.scala:204) as input to shuffle 7
24/03/26 16:14:10 INFO DAGScheduler: Got map stage job 30 (collect at StringIndexer.scala:204) with 1 output partitions
24/03/26 16:14:10 INFO DAGScheduler: Final stage: ShuffleMapStage 37 (collect at StringIndexer.scala:204)
24/03/26 16:14:10 INFO DAGScheduler: Parents of final stage: List()
24/03/26 16:14:10 INFO DAGScheduler: Missing parents: List()
24/03/26 16:14:10 INFO DAGScheduler: Submitting ShuffleMapStage 37 (MapPartitionsRDD[140] at collect at StringIndexer.scala:204), which has no missing parents
24/03/26 16:14:10 INFO MemoryStore: Block broadcast_36 stored as values in memory (estimated size 32.5 KiB, free 433.2 MiB)
24/03/26 16:14:10 INFO MemoryStore: Block broadcast_36_piece0 stored as bytes in memory (estimated size 15.2 KiB, free 433.2 MiB)
24/03/26 16:14:10 INFO BlockManagerInfo: Added broadcast_36_piece0 in memory on b2edbf16b081:40551 (size: 15.2 KiB, 

+---+-------------+------------+-------------+------------+-----------+------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|species_indx|
+---+-------------+------------+-------------+------------+-----------+------------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|         0.0|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|         0.0|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|         0.0|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|         0.0|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|         0.0|
+---+-------------+------------+-------------+------------+-----------+------------+
only showing top 5 rows



24/03/26 16:14:11 INFO CodeGenerator: Code generated in 21.656638 ms
24/03/26 16:14:11 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
24/03/26 16:14:11 INFO DAGScheduler: Got job 32 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
24/03/26 16:14:11 INFO DAGScheduler: Final stage: ResultStage 40 (showString at NativeMethodAccessorImpl.java:0)
24/03/26 16:14:11 INFO DAGScheduler: Parents of final stage: List()
24/03/26 16:14:11 INFO DAGScheduler: Missing parents: List()
24/03/26 16:14:11 INFO DAGScheduler: Submitting ResultStage 40 (MapPartitionsRDD[148] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
24/03/26 16:14:11 INFO MemoryStore: Block broadcast_38 stored as values in memory (estimated size 38.9 KiB, free 433.1 MiB)
24/03/26 16:14:11 INFO MemoryStore: Block broadcast_38_piece0 stored as bytes in memory (estimated size 18.0 KiB, free 433.1 MiB)
24/03/26 16:14:11 INFO BlockManagerInfo: Added broa

We are going to do a classic data select of the data,  much like an sql select, to reformat the data a bit, reordering columns and removing
the text species name

In [50]:
#creating a seperate dataframe with re-ordered columns
df = data.select("species_indx","SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm")

#Inspect the dataframe
df.show(5)

+------------+-------------+------------+-------------+------------+
|species_indx|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|
+------------+-------------+------------+-------------+------------+
|         0.0|          5.1|         3.5|          1.4|         0.2|
|         0.0|          4.9|         3.0|          1.4|         0.2|
|         0.0|          4.7|         3.2|          1.3|         0.2|
|         0.0|          4.6|         3.1|          1.5|         0.2|
|         0.0|          5.0|         3.6|          1.4|         0.2|
+------------+-------------+------------+-------------+------------+
only showing top 5 rows



24/03/26 16:16:49 INFO CodeGenerator: Code generated in 13.251362 ms
24/03/26 16:16:49 INFO SparkContext: Starting job: showString at <unknown>:0
24/03/26 16:16:49 INFO DAGScheduler: Got job 33 (showString at <unknown>:0) with 1 output partitions
24/03/26 16:16:49 INFO DAGScheduler: Final stage: ResultStage 41 (showString at <unknown>:0)
24/03/26 16:16:49 INFO DAGScheduler: Parents of final stage: List()
24/03/26 16:16:49 INFO DAGScheduler: Missing parents: List()
24/03/26 16:16:49 INFO DAGScheduler: Submitting ResultStage 41 (MapPartitionsRDD[153] at showString at <unknown>:0), which has no missing parents
24/03/26 16:16:49 INFO MemoryStore: Block broadcast_39 stored as values in memory (estimated size 38.4 KiB, free 433.1 MiB)
24/03/26 16:16:49 INFO MemoryStore: Block broadcast_39_piece0 stored as bytes in memory (estimated size 17.9 KiB, free 433.0 MiB)
24/03/26 16:16:49 INFO BlockManagerInfo: Added broadcast_39_piece0 in memory on b2edbf16b081:40551 (size: 17.9 KiB, free: 433.8 MiB

We are now going to rearrange the data, mapping it as the first column (our target) and then a Spark Dense Vector

In [51]:
# Define the `input_data` as Dense Vector
input_data = df.rdd.map(lambda x: (x[0], DenseVector(x[1:])))

24/03/26 16:16:57 INFO CodeGenerator: Code generated in 6.876369 ms


Create a new data frame, using the target as the index, and the DenseVector as the features

In [52]:
# Creating a new Indexed Dataframe
df_indx = spark.createDataFrame(input_data, ["label", "features"])

24/03/26 16:17:02 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/03/26 16:17:02 INFO DAGScheduler: Got job 34 (runJob at PythonRDD.scala:181) with 1 output partitions
24/03/26 16:17:02 INFO DAGScheduler: Final stage: ResultStage 42 (runJob at PythonRDD.scala:181)
24/03/26 16:17:02 INFO DAGScheduler: Parents of final stage: List()
24/03/26 16:17:02 INFO DAGScheduler: Missing parents: List()
24/03/26 16:17:02 INFO DAGScheduler: Submitting ResultStage 42 (PythonRDD[161] at RDD at PythonRDD.scala:53), which has no missing parents
24/03/26 16:17:02 INFO MemoryStore: Block broadcast_40 stored as values in memory (estimated size 45.7 KiB, free 433.0 MiB)
24/03/26 16:17:02 INFO MemoryStore: Block broadcast_40_piece0 stored as bytes in memory (estimated size 21.7 KiB, free 433.0 MiB)
24/03/26 16:17:02 INFO BlockManagerInfo: Added broadcast_40_piece0 in memory on b2edbf16b081:40551 (size: 21.7 KiB, free: 433.8 MiB)
24/03/26 16:17:02 INFO SparkContext: Created broadcast 40 from

In [53]:
#view the indexed dataframe
df_indx.show(5)

24/03/26 16:17:07 INFO CodeGenerator: Code generated in 41.281984 ms
24/03/26 16:17:07 INFO BlockManagerInfo: Removed broadcast_31_piece0 on b2edbf16b081:40551 in memory (size: 9.9 KiB, free: 433.8 MiB)
24/03/26 16:17:07 INFO SparkContext: Starting job: showString at <unknown>:0
24/03/26 16:17:07 INFO DAGScheduler: Got job 35 (showString at <unknown>:0) with 1 output partitions
24/03/26 16:17:07 INFO DAGScheduler: Final stage: ResultStage 43 (showString at <unknown>:0)
24/03/26 16:17:07 INFO DAGScheduler: Parents of final stage: List()
24/03/26 16:17:07 INFO DAGScheduler: Missing parents: List()
24/03/26 16:17:07 INFO DAGScheduler: Submitting ResultStage 43 (MapPartitionsRDD[167] at showString at <unknown>:0), which has no missing parents
24/03/26 16:17:07 INFO BlockManagerInfo: Removed broadcast_38_piece0 on b2edbf16b081:40551 in memory (size: 18.0 KiB, free: 433.8 MiB)
24/03/26 16:17:07 INFO MemoryStore: Block broadcast_41 stored as values in memory (estimated size 52.1 KiB, free 433

+-----+-----------------+
|label|         features|
+-----+-----------------+
|  0.0|[5.1,3.5,1.4,0.2]|
|  0.0|[4.9,3.0,1.4,0.2]|
|  0.0|[4.7,3.2,1.3,0.2]|
|  0.0|[4.6,3.1,1.5,0.2]|
|  0.0|[5.0,3.6,1.4,0.2]|
+-----+-----------------+
only showing top 5 rows



24/03/26 16:17:08 INFO Executor: Finished task 0.0 in stage 43.0 (TID 40). 2268 bytes result sent to driver
24/03/26 16:17:08 INFO TaskSetManager: Finished task 0.0 in stage 43.0 (TID 40) in 1055 ms on b2edbf16b081 (executor driver) (1/1)
24/03/26 16:17:08 INFO TaskSchedulerImpl: Removed TaskSet 43.0, whose tasks have all completed, from pool 
24/03/26 16:17:08 INFO DAGScheduler: ResultStage 43 (showString at <unknown>:0) finished in 1.067 s
24/03/26 16:17:08 INFO DAGScheduler: Job 35 is finished. Cancelling potential speculative or zombie tasks for this job
24/03/26 16:17:08 INFO TaskSchedulerImpl: Killing all running tasks in stage 43: Stage finished
24/03/26 16:17:08 INFO DAGScheduler: Job 35 finished: showString at <unknown>:0, took 1.075178 s


We will fit some ML models, so we need to standardize the features

In [54]:
#Initialize Standard Scaler
stdScaler = StandardScaler(inputCol="features", outputCol="features_scaled")

#Fit the Standard Scaler to the indexed Dataframe
scaler = stdScaler.fit(df_indx)

#Transform the dataframe
df_scaled =scaler.transform(df_indx)

24/03/26 16:17:15 INFO CodeGenerator: Code generated in 6.918969 ms
24/03/26 16:17:15 INFO DAGScheduler: Registering RDD 170 (first at StandardScaler.scala:113) as input to shuffle 8
24/03/26 16:17:15 INFO DAGScheduler: Got map stage job 36 (first at StandardScaler.scala:113) with 1 output partitions
24/03/26 16:17:15 INFO DAGScheduler: Final stage: ShuffleMapStage 44 (first at StandardScaler.scala:113)
24/03/26 16:17:15 INFO DAGScheduler: Parents of final stage: List()
24/03/26 16:17:15 INFO DAGScheduler: Missing parents: List()
24/03/26 16:17:15 INFO DAGScheduler: Submitting ShuffleMapStage 44 (MapPartitionsRDD[170] at first at StandardScaler.scala:113), which has no missing parents
24/03/26 16:17:15 INFO MemoryStore: Block broadcast_42 stored as values in memory (estimated size 60.7 KiB, free 433.5 MiB)
24/03/26 16:17:15 INFO MemoryStore: Block broadcast_42_piece0 stored as bytes in memory (estimated size 27.8 KiB, free 433.4 MiB)
24/03/26 16:17:15 INFO BlockManagerInfo: Added broad

In [55]:
#Viewing the Scaled Data
df_scaled.show(5)

24/03/26 16:17:20 INFO CodeGenerator: Code generated in 8.475077 ms
24/03/26 16:17:20 INFO SparkContext: Starting job: showString at <unknown>:0
24/03/26 16:17:20 INFO DAGScheduler: Got job 38 (showString at <unknown>:0) with 1 output partitions
24/03/26 16:17:20 INFO DAGScheduler: Final stage: ResultStage 47 (showString at <unknown>:0)
24/03/26 16:17:20 INFO DAGScheduler: Parents of final stage: List()
24/03/26 16:17:20 INFO DAGScheduler: Missing parents: List()
24/03/26 16:17:20 INFO DAGScheduler: Submitting ResultStage 47 (MapPartitionsRDD[176] at showString at <unknown>:0), which has no missing parents
24/03/26 16:17:20 INFO MemoryStore: Block broadcast_44 stored as values in memory (estimated size 57.9 KiB, free 433.3 MiB)
24/03/26 16:17:20 INFO MemoryStore: Block broadcast_44_piece0 stored as bytes in memory (estimated size 26.3 KiB, free 433.3 MiB)
24/03/26 16:17:20 INFO BlockManagerInfo: Added broadcast_44_piece0 in memory on b2edbf16b081:40551 (size: 26.3 KiB, free: 433.9 MiB)

+-----+-----------------+--------------------+
|label|         features|     features_scaled|
+-----+-----------------+--------------------+
|  0.0|[5.1,3.5,1.4,0.2]|[6.15892840883878...|
|  0.0|[4.9,3.0,1.4,0.2]|[5.9174018045706,...|
|  0.0|[4.7,3.2,1.3,0.2]|[5.67587520030241...|
|  0.0|[4.6,3.1,1.5,0.2]|[5.55511189816831...|
|  0.0|[5.0,3.6,1.4,0.2]|[6.03816510670469...|
+-----+-----------------+--------------------+
only showing top 5 rows



24/03/26 16:17:20 INFO CodeGenerator: Code generated in 6.390567 ms
24/03/26 16:17:20 INFO CodeGenerator: Code generated in 25.887627 ms
24/03/26 16:17:20 INFO Executor: Finished task 0.0 in stage 47.0 (TID 43). 2652 bytes result sent to driver
24/03/26 16:17:20 INFO TaskSetManager: Finished task 0.0 in stage 47.0 (TID 43) in 346 ms on b2edbf16b081 (executor driver) (1/1)
24/03/26 16:17:20 INFO TaskSchedulerImpl: Removed TaskSet 47.0, whose tasks have all completed, from pool 
24/03/26 16:17:20 INFO DAGScheduler: ResultStage 47 (showString at <unknown>:0) finished in 0.362 s
24/03/26 16:17:20 INFO DAGScheduler: Job 38 is finished. Cancelling potential speculative or zombie tasks for this job
24/03/26 16:17:20 INFO TaskSchedulerImpl: Killing all running tasks in stage 47: Stage finished
24/03/26 16:17:20 INFO DAGScheduler: Job 38 finished: showString at <unknown>:0, took 0.370365 s


In [56]:
#Dropping the Features column
df_scaled = df_scaled.drop("features")

Now Create the test and train sets

In [57]:
train_data, test_data = df_scaled.randomSplit([0.9, 0.1], seed = 12345)

In [58]:
#Inspect Training Data
train_data.show(5)

24/03/26 16:17:39 INFO CodeGenerator: Code generated in 18.898072 ms
24/03/26 16:17:39 INFO SparkContext: Starting job: showString at <unknown>:0
24/03/26 16:17:39 INFO DAGScheduler: Got job 39 (showString at <unknown>:0) with 1 output partitions
24/03/26 16:17:39 INFO DAGScheduler: Final stage: ResultStage 48 (showString at <unknown>:0)
24/03/26 16:17:39 INFO DAGScheduler: Parents of final stage: List()
24/03/26 16:17:39 INFO DAGScheduler: Missing parents: List()
24/03/26 16:17:39 INFO DAGScheduler: Submitting ResultStage 48 (MapPartitionsRDD[178] at showString at <unknown>:0), which has no missing parents
24/03/26 16:17:39 INFO MemoryStore: Block broadcast_45 stored as values in memory (estimated size 69.4 KiB, free 433.2 MiB)
24/03/26 16:17:39 INFO MemoryStore: Block broadcast_45_piece0 stored as bytes in memory (estimated size 31.0 KiB, free 433.2 MiB)
24/03/26 16:17:39 INFO BlockManagerInfo: Added broadcast_45_piece0 in memory on b2edbf16b081:40551 (size: 31.0 KiB, free: 433.9 MiB

+-----+--------------------+
|label|     features_scaled|
+-----+--------------------+
|  0.0|[5.19282199176603...|
|  0.0|[5.31358529390013...|
|  0.0|[5.31358529390013...|
|  0.0|[5.31358529390013...|
|  0.0|[5.43434859603422...|
+-----+--------------------+
only showing top 5 rows



24/03/26 16:17:40 INFO PythonRunner: Times: total = 1054, boot = 6, init = 1043, finish = 5
24/03/26 16:17:40 INFO Executor: Finished task 0.0 in stage 48.0 (TID 44). 2930 bytes result sent to driver
24/03/26 16:17:40 INFO TaskSetManager: Finished task 0.0 in stage 48.0 (TID 44) in 1114 ms on b2edbf16b081 (executor driver) (1/1)
24/03/26 16:17:40 INFO TaskSchedulerImpl: Removed TaskSet 48.0, whose tasks have all completed, from pool 
24/03/26 16:17:40 INFO DAGScheduler: ResultStage 48 (showString at <unknown>:0) finished in 1.126 s
24/03/26 16:17:40 INFO DAGScheduler: Job 39 is finished. Cancelling potential speculative or zombie tasks for this job
24/03/26 16:17:40 INFO TaskSchedulerImpl: Killing all running tasks in stage 48: Stage finished
24/03/26 16:17:40 INFO DAGScheduler: Job 39 finished: showString at <unknown>:0, took 1.130693 s


We will run a series of models,   this is the list of model names and storage for the results

In [59]:
model = ['Decision Tree','Random Forest','Naive Bayes']
model_results = []

Build a descion tree classifier

Note the interface style,  declare the model, setting parameters,  then train and then apply to the test data

Looks kinda familiar at this point

In [60]:
# -- Decision Tree Classifier --

dtc = DecisionTreeClassifier(labelCol="label", featuresCol="features_scaled")          #instantiate the model
dtc_model = dtc.fit(train_data)                                                        #train the model
dtc_pred = dtc_model.transform(test_data)                                              #model predictions

#Evaluate the Model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
dtc_acc = evaluator.evaluate(dtc_pred)
#print("Decision Tree Classifier Accuracy =", '{:.2%}'.format(dtc_acc))
model_results.extend([[model[0],'{:.2%}'.format(dtc_acc)]])                               #appending to list

24/03/26 16:17:54 INFO Instrumentation: [35215bcb] Stage class: DecisionTreeClassifier
24/03/26 16:17:54 INFO Instrumentation: [35215bcb] Stage uid: DecisionTreeClassifier_1765d766a156
24/03/26 16:17:54 INFO CodeGenerator: Code generated in 18.751542 ms
24/03/26 16:17:54 INFO Instrumentation: [35215bcb] training: numPartitions=1 storageLevel=StorageLevel(1 replicas)
24/03/26 16:17:54 INFO CodeGenerator: Code generated in 26.085807 ms
24/03/26 16:17:54 INFO DAGScheduler: Registering RDD 184 (take at DatasetUtils.scala:193) as input to shuffle 9
24/03/26 16:17:54 INFO DAGScheduler: Got map stage job 40 (take at DatasetUtils.scala:193) with 1 output partitions
24/03/26 16:17:54 INFO DAGScheduler: Final stage: ShuffleMapStage 49 (take at DatasetUtils.scala:193)
24/03/26 16:17:54 INFO DAGScheduler: Parents of final stage: List()
24/03/26 16:17:54 INFO DAGScheduler: Missing parents: List()
24/03/26 16:17:54 INFO DAGScheduler: Submitting ShuffleMapStage 49 (MapPartitionsRDD[184] at take at Da

Here's a random forest.

Only 10 trees?  Just saving time I suppose, the IRIS set is easy

In [61]:
# -- Random Forest Classifier --

rfc = RandomForestClassifier(labelCol="label", featuresCol="features_scaled", numTrees=10)          #instantiate the model
rfc_model = rfc.fit(train_data)                                                                     #train the model
rfc_pred = rfc_model.transform(test_data)                                                           #model predictions

#Evaluate the Model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
rfc_acc = evaluator.evaluate(rfc_pred)
#print("Random Forest Classifier Accuracy =", '{:.2%}'.format(rfc_acc))
model_results.extend([[model[1],'{:.2%}'.format(rfc_acc)]])                                            #appending to list

24/03/26 16:18:10 INFO Instrumentation: [36166e1c] Stage class: RandomForestClassifier
24/03/26 16:18:10 INFO Instrumentation: [36166e1c] Stage uid: RandomForestClassifier_8849dfebca3c
24/03/26 16:18:10 INFO Instrumentation: [36166e1c] training: numPartitions=1 storageLevel=StorageLevel(1 replicas)
24/03/26 16:18:10 INFO DAGScheduler: Registering RDD 228 (take at DatasetUtils.scala:193) as input to shuffle 17
24/03/26 16:18:10 INFO DAGScheduler: Got map stage job 51 (take at DatasetUtils.scala:193) with 1 output partitions
24/03/26 16:18:10 INFO DAGScheduler: Final stage: ShuffleMapStage 68 (take at DatasetUtils.scala:193)
24/03/26 16:18:10 INFO DAGScheduler: Parents of final stage: List()
24/03/26 16:18:10 INFO DAGScheduler: Missing parents: List()
24/03/26 16:18:10 INFO DAGScheduler: Submitting ShuffleMapStage 68 (MapPartitionsRDD[228] at take at DatasetUtils.scala:193), which has no missing parents
24/03/26 16:18:10 INFO MemoryStore: Block broadcast_70 stored as values in memory (es

This is a method we haven't seen yet,  a Naive Bayes Classifer

In [62]:
# -- Naive Bayes Classifier --

nbc = NaiveBayes(smoothing=1.0,modelType="multinomial", labelCol="label",featuresCol="features_scaled")    #instantiate the model
nbc_model = nbc.fit(train_data)                                                                          #train the model
nbc_pred = nbc_model.transform(test_data)                                                                #model predictions

#Evaluate the Model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
nbc_acc = evaluator.evaluate(nbc_pred)
#print("Naive Bayes Accuracy =", '{:.2%}'.format(nbc_acc))
model_results.extend([[model[2],'{:.2%}'.format(nbc_acc)]])                                            #appending to list

24/03/26 16:18:26 INFO Instrumentation: [dd919d40] Stage class: NaiveBayes
24/03/26 16:18:26 INFO Instrumentation: [dd919d40] Stage uid: NaiveBayes_00f08a0a0bc3
24/03/26 16:18:26 INFO Instrumentation: [dd919d40] training: numPartitions=1 storageLevel=StorageLevel(1 replicas)
24/03/26 16:18:26 INFO Instrumentation: [dd919d40] {"labelCol":"label","featuresCol":"features_scaled","modelType":"multinomial","smoothing":1.0}
24/03/26 16:18:26 INFO CodeGenerator: Code generated in 39.094439 ms
24/03/26 16:18:27 INFO DAGScheduler: Registering RDD 278 (collect at NaiveBayes.scala:222) as input to shuffle 25
24/03/26 16:18:27 INFO DAGScheduler: Got map stage job 62 (collect at NaiveBayes.scala:222) with 1 output partitions
24/03/26 16:18:27 INFO DAGScheduler: Final stage: ShuffleMapStage 87 (collect at NaiveBayes.scala:222)
24/03/26 16:18:27 INFO DAGScheduler: Parents of final stage: List()
24/03/26 16:18:27 INFO DAGScheduler: Missing parents: List()
24/03/26 16:18:27 INFO DAGScheduler: Submittin

There is an explicity "garbage collection" call to free memory that is not being used.   Python has automatic garbage collection,  but it is being used explictly here, this may be Spark-related, but I'm not sure right now

Java is big on garbage collection and Spark runs in a Java environment

In [63]:
#freeing memory
gc.collect()

3743

Here is the output of the model results

tabulate() looks handy, it looks like it was loaded from a package called (wait for it!) tabulate   

In [64]:
print (tabulate(model_results, headers=["Classifier Models", "Accuracy"]))

Classifier Models    Accuracy
-------------------  ----------
Decision Tree        90.91%
Random Forest        100.00%
Naive Bayes          100.00%
